# Cirrus Demo

## Simple Example

This will run a simple logistic regression on the Criteo Kaggle dataset

In [1]:
import collab
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import sys
print(sys.version)

print __version__ # requires version >= 1.9.0

2.7.12 (default, Dec  4 2017, 14:50:18) 
[GCC 5.4.0 20160609]
2.7.0


In [2]:

import numpy as np 
import plotly.plotly as py  
import plotly.tools as tls   
import plotly.graph_objs as go

init_notebook_mode(connected=True)
stream_ids = tls.get_credentials_file()['stream_ids']

# Get stream id from stream id list 
stream_id = stream_ids[0]


py.iplot([{'x': [], 'y': [], 'type': 'scatter', 'mode': 'lines+markers',
            'stream': {'token': stream_id, 'maxpoints': 80}
          }],
        filename='Time Series', fileopt='overwrite')

In [3]:

# We will provide the stream link object the same token that's associated with the trace we wish to stream to
s = py.Stream(stream_id)

# We then open a connection
s.open()

In [4]:
def progress_callback(time_loss, cost, task):
    print("Current training loss:", time_loss, "current cost ($): ", cost)
    s.write(dict(x=time_loss[0], y=time_loss[1]))



In [ ]:
data_bucket = 'cirrus-criteo-kaggle-19b-random'
model = 'model_v1'

lr_task = collab.CollaborativeFiltering(
             # number of workers and number of PSs
             n_workers = 3, n_ps = 2,
             # path to s3 bucket with input dataset
             dataset = data_bucket,
             # sgd update LR and epsilon
             learning_rate=0.01, epsilon=0.0001,
             #
             progress_callback = progress_callback,
             # stop workload after these many seconds
             timeout = 0,
             # stop workload once we reach this loss
             threshold_loss=0,
             # resume execution from model stored in this s3 bucket
             resume_model = model,
             # aws key name
             key_name='mykey',
             # path to aws key
             key_path='/home/camus/Downloads/mykey.pem',
             # ip where ps lives
             ps_ip='ec2-34-219-23-178.us-west-2.compute.amazonaws.com',
             # username of VM
             ps_username='ec2-user',
             # choose between adagrad, sgd, nesterov, momentum
             opt_method = 'adagrad',
             # checkpoint model every x secs
             checkpoint_model = 60,
             #
             minibatch_size=20,
             # model size
             model_bits=19,
             # whether to filter gradient weights
             use_grad_threshold=True,
             # threshold value
             grad_threshold=0.001,
             # range of training minibatches
             train_set=(0,824),
             # range of testing minibatches
             test_set=(825,827)
             )

lr_task.run()

#model, loss = lr_task.wait()

Running collaborative filtering workload
Starting CollaborativeFilteringTask
User's specific ip: ec2-34-219-23-178.us-west-2.compute.amazonaws.com
Copying ps to vm
Done waiting... Attempting to copy over binary
Copied parameter server
Defining configuration file

input_path: /home/ec2-user/cirrus/examples/ml/tests/test_mf/nf_parsed 
input_type: csv 
minibatch_size: 20 
s3_size: 10000 
model_type: CollaborativeFiltering 
learning_rate: 0.010000 
epsilon: 0.000100 
normalize: 1 
limit_samples: 50000000 
use_bias: 1 
num_users: 480189 
num_items: 17770 
train_set: 0-5 
s3_bucket: cirrus-netflix-not-normalized

Launching ps
Launching parameter server
('cmd:', 'ssh -o "StrictHostKeyChecking no" -i /home/camus/Downloads/mykey.pem ec2-user@ec2-34-219-23-178.us-west-2.compute.amazonaws.com "nohup ./parameter_server config_cf.txt 10000 1 &> ps_output &"')
Launching lambdas
launching lambda with id 3
launching lambda with id 4
('cmd', 'ssh -o "StrictHostKeyChecking no" -i /home/camus/Downloads/m

In [ ]:
s.close